

> Mounting **My Drive** from Google Drive [train and test data is there]



In [0]:
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).




> Getting train and test data from My Drive



In [0]:
from keras.preprocessing.image import ImageDataGenerator

IMAGES_PATH = '/content/gdrive/My Drive/dataset/images/'
TRAIN_PATH = IMAGES_PATH + 'train'
TEST_PATH = IMAGES_PATH + 'test'

data_generator = ImageDataGenerator(rescale = 1.0 / 255)

train_generator = data_generator.flow_from_directory(TRAIN_PATH,
                                              batch_size = 800,
                                              class_mode = None)

test_generator = data_generator.flow_from_directory(TEST_PATH,
                                              batch_size = 200,
                                              class_mode = None)

Found 800 images belonging to 2 classes.
Found 200 images belonging to 2 classes.



> Preparing train and test data

*   **input** consists of **L*** channel
*   **output** consists of **a*** and **b*** channels
*   **a*** and **b*** channels' values range from -127 to 128, that is why we need to divide them by 128, thereby normalizing their values to -1 and 1




In [0]:
from skimage.color import rgb2lab, lab2rgb
from skimage.io import imshow
import matplotlib.pyplot as plt
import numpy as np

train_input = [(rgb2lab(img)[:,:,0]) for img in train_generator[0]]
train_target = [(rgb2lab(img)[:,:,1:] / 128) for img in train_generator[0]]

test_input = [(rgb2lab(img)[:,:,0]) for img in test_generator[0]]
test_target = [(rgb2lab(img)[:,:,1:]) for img in test_generator[0]]

train_input = np.array(train_input)
train_target = np.array(train_target)
train_input = train_input.reshape(train_input.shape + (1, ))

test_input = np.array(test_input)
test_target = np.array(test_target)
test_input = test_input.reshape(test_input.shape + (1, ))



> Method to combine **L a b** channels and convert to **RGB**



In [0]:
def lab_to_rgb(input, output):
  rgb_images = []

  for x, y in zip(input, output):
    out = np.zeros((256, 256, 3))
    out[:,:,0] = x[:,:,0]
    out[:,:,1:] = y
    rgb_images.append(lab2rgb(out))

  return rgb_images



> Method to plot specific **images array**



In [0]:
def imshow_rgb(images):
  for img in images:
    plt.imshow(img)
    plt.show()



> **ImageColorizer** class definiton,

*   **encoder** and **decoder** blocks were created for convenience (code reuse)
*   **autoencoder** nn is defined with **tanh** as output function
*   **tanh** is used as an activation function, since we need values of range from -1 to 1 (normalized range of **a*** and **b*** channels)





In [0]:
from keras.layers import Conv2D, MaxPooling2D, Activation, Input, UpSampling2D, LeakyReLU
from keras.models import Model

class ImageColorizer():
  def __init__(self):
    self.input_shape = (256, 256, 1)    # L channel
    self.output_channels = 2            # A and B channels
    self.kernel_size = 3
    self.padding = 'same'

  # convolutional block for encoder
  def encoder_block(self, n_filters, input_layer, is_downsampled = False):
    encoder = Conv2D(n_filters, kernel_size = self.kernel_size, padding = self.padding)(input_layer)
    if is_downsampled: 
      encoder = MaxPooling2D()(encoder)
      # encoder = Conv2D(n_filters, kernel_size = self.kernel_size, padding = self.padding, strides = 2)(input_layer)
    encoder = Activation('relu')(encoder)
    return encoder

  # convolutional block for decoder
  def decoder_block(self, n_filters, input_layer, is_maxpooled = False):
    decoder = Conv2D(n_filters, kernel_size = self.kernel_size, padding = self.padding)(input_layer)
    if is_maxpooled:
      decoder = UpSampling2D()(decoder)
    decoder = Activation('relu')(decoder)
    return decoder

  def build_autoencoder(self):
    input_layer = Input(shape = self.input_shape)

    # encoder network with input = 256 x 256 x 1
    encoder_1 = self.encoder_block(64, input_layer, is_downsampled = True)                      # 128 x 128 x 64
    encoder_2 = self.encoder_block(128, encoder_1)                                              # 128 x 128 x 128
    encoder_3 = self.encoder_block(128, encoder_2, is_downsampled = True)                       # 64 x 64 x 128
    encoder_4 = self.encoder_block(256, encoder_3)                                              # 64 x 64 x 256
    encoder_5 = self.encoder_block(256, encoder_4, is_downsampled = True)                       # 32 x 32 x 256
    encoder_6 = self.encoder_block(512, encoder_5)                                              # 32 x 32 x 512
    encoder_7 = self.encoder_block(512, encoder_6)                                              # 32 x 32 x 512
    encoder_8 = self.encoder_block(256, encoder_7)                                              # 32 x 32 x 256


    # decoder network
    decoder_1 = self.decoder_block(128, encoder_8, is_maxpooled = True)                         # 64 x 64 x 128 
    decoder_2 = self.decoder_block(64, decoder_1)                                               # 64 x 64 x 64
    decoder_3 = self.decoder_block(32, decoder_2, is_maxpooled = True)                          # 128 x 128 x 32
    decoder_4 = self.decoder_block(16, decoder_3)                                               # 128 x 128 x 16
    

    decoder_5 = Conv2D(self.output_channels, kernel_size = self.kernel_size, padding = self.padding)(decoder_4)    # 256 x 256 x 2
    decoder_5 = UpSampling2D()(decoder_5)
    output_layer = Activation('tanh')(decoder_5) 

    model = Model(input_layer, output_layer)
    return model


> **Main method** definition



In [0]:
import tensorflow as tf

if __name__ == '__main__':
  colorizer = ImageColorizer()
  model = colorizer.build_autoencoder()
  # model.summary()
  # model.compile(optimizer = 'Adam', loss = 'mse', metrics = ['accuracy'])
  # history = model.fit(train_input, train_target, validation_split = 0.05, epochs = 100, batch_size = 32, shuffle = True)
  # history = model.fit(train_input, train_target, validation_split = 0.05, epochs = 500, batch_size = 32, shuffle = True)
  # history = model.fit(train_input, train_target, validation_split = 0.05, epochs = 700, batch_size = 32, shuffle = True)

  # model.save('/content/gdrive/My Drive/dataset/autoencoder.model')
  # model.save('/content/gdrive/My Drive/dataset/autoencoder100.model')
  # model.save('/content/gdrive/My Drive/dataset/autoencoder700.model')

  model = tf.keras.models.load_model('/content/gdrive/My Drive/dataset/autoencoder700.model')

  result = model.predict(test_input)
  result = result * 128                 # values in range from -1 to 1 due to tanh activation function



> Saving the **history** object in the specified file


In [0]:
import pickle

ROOT_PATH = '/content/gdrive/My Drive/dataset/'

def save_history(history_filename):
  f = open(ROOT_PATH + history_filename, 'wb')
  pickle.dump(history.history, f)
  f.close()

# save_history('history100.pckl')



> Uploading the **history** object from the specified file



In [0]:
def load_history(history_filename):
  f = open(ROOT_PATH + history_filename, 'rb')
  history = pickle.load(f)
  f.close()
  return history

# history = load_history('history100.pckl')



> Plotting training and validation loss values



In [0]:
def plot_loss(history):
  plt.figure(figsize=(12,6))
  plt.plot(history['loss'])
  plt.plot(history['val_loss'])
  plt.title('Model loss')
  plt.ylabel('Loss')
  plt.xlabel('Epoch')
  plt.legend(['Train', 'Test'], loc='upper left')
  plt.show()

plot_loss(history)



> Plot training and validation accuracy values



In [0]:
def plot_accuracy(history):
  plt.figure(figsize=(12,6))
  plt.plot(history['accuracy'])
  plt.plot(history['val_accuracy'])
  plt.title('Model accuracy')
  plt.ylabel('Accuracy')
  plt.xlabel('Epoch')
  plt.legend(['Train', 'Test'], loc='upper left')
  plt.show()

plot_accuracy(history)



> Function to **compare** greyscale image, resulting image and real image


In [0]:
def performance_result(output):
  greyscale_images = lab_to_rgb(test_input, np.zeros(output.shape))
  output_images = lab_to_rgb(test_input, output)
  real_images = lab_to_rgb(test_input, test_target)
  images = greyscale_images + output_images + real_images

  for gi, oi, ri in zip(greyscale_images, output_images, real_images):
    plt.subplot(1, 3, 1)
    plt.imshow(gi)

    plt.subplot(1, 3, 2)
    plt.imshow(oi)

    plt.subplot(1, 3, 3)
    plt.imshow(ri)

    plt.show()

    
performance_result(result)